In [2]:
from influxdb import InfluxDBClient
from kafka import KafkaConsumer,TopicPartition
import json
import time
import os
import pandas as pd

import time
import logging
from jaeger_client import Config
from opentracing_instrumentation.request_context import get_current_span, span_in_context

def influxdb_connect(domain, port, database_name):
    print ("Trying to connect to InfluxDB server without proxy:", domain, "on port:", port)
    proxies = { "http": None, "https": None}
    try:
        client = InfluxDBClient(host=domain, 
                                port=port, 
                                database=database_name,
                                proxies=proxies)
        print("connection sucess!")
        return client
    except:
        print("connection error!")
        return 

def init_tracer(service):
    logging.getLogger('').handlers = []
    logging.basicConfig(format='%(message)s', level=logging.DEBUG)    
    config = Config(
        config={
            'sampler': {
                'type': 'const',
                'param': 1,
            },
            'logging': True,
        },
        service_name=service,
    )
    if config._initialized == False :
        return config.initialize_tracer()
    else :
        print("error : create new tracer...")
        return config.new_tracer()    
    
def consumer_single_topic():
    consumer = KafkaConsumer(bootstrap_servers='localhost:9092')
    topic="eolienne_jour_1"
    tp = TopicPartition(topic,0)
    consumer.assign([tp])
    consumer.seek_to_end(tp)
    lastOffset = consumer.position(tp)
    print(lastOffset)
    consumer.seek_to_beginning(tp) 
    #end
    list_data=[]
    with tracer.start_span('collect_data',child_of=get_current_span()) as span:
        span.set_tag('kafka','collect_data_jour_1')
        with span_in_context(span):
            for message in consumer:
                msg = message.value.decode()
                influxdata={
                    "measurement": "smartGrid",
                    "tags": {
                        "tag_1": "eolienne"
                    },
                    "time": msg.split(";")[0].split(":",1)[1],
                    "fields": {
                        "count": msg.split(":",1)[0],
                        "message": msg.split(";",2)[2]
                    }
                }
                list_data.append(influxdata)
                if message.offset == lastOffset - 1:
                    print(message.offset)
                    break
                
            consumer.close()
            return list_data
        
def insert_bulk(client, policy, list_data):
    with tracer.start_span('insert_bulk',child_of=get_current_span()) as span:
        span.set_tag('influxdb','operation:insert_bulk')
        with span_in_context(span):
            try :
                client.write_points(list_data,retention_policy=policy)
                return "sucess"
            except : 
                return "error"
            
def test_insert_bulk(client, policy):
    with tracer.start_span('test_bulk_insertion') as span:
        span.set_tag('influxdb','insertion_test')
        with span_in_context(span):
            data=consumer_single_topic()
            result=insert_bulk(client, policy, data)
            return result
    
if __name__=="__main__":
    database = "test"
    policy_name = "test_policy"
    client = influxdb_connect("localhost", 9086,database)
    client.create_retention_policy(name=policy_name, 
                                   duration="INF", 
                                   database="test",
                                   replication="1", 
                                   default=False,
                                   shard_duration="1h50m")
    tracer = init_tracer('influxdb_test_insert_eolienne_1_jour')
    result = test_insert_bulk(client, policy_name)

Starting new HTTP connection (1): localhost:9086
http://localhost:9086 "POST /query?q=CREATE+RETENTION+POLICY+%22test_policy%22+ON+%22test%22+DURATION+INF+REPLICATION+1+SHARD+DURATION+1h50m&db=test HTTP/1.1" 200 None
Initializing Jaeger Tracer with UDP reporter
Using selector: EpollSelector
Using sampler ConstSampler(True)
Added sensor with name connections-closed
Added sensor with name connections-created
Added sensor with name select-time
Added sensor with name io-time
Initiating connection to node bootstrap-0 at localhost:9092
Added sensor with name bytes-sent-received
Added sensor with name bytes-sent
Added sensor with name bytes-received
Added sensor with name request-latency
Added sensor with name node-bootstrap-0.bytes-sent
Added sensor with name node-bootstrap-0.bytes-received
Added sensor with name node-bootstrap-0.latency
<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <disconnected> [unspecified None]>: creating new socket
<BrokerConnection node_id=bootstrap-0 host

Trying to connect to InfluxDB server without proxy: localhost on port: 9086
connection sucess!
error : create new tracer...


Sending request MetadataRequest_v0(topics=[])
<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]> Request 2: MetadataRequest_v0(topics=[])
Received correlation id: 1
Processing response ApiVersionResponse_v0
Received correlation id: 2
Processing response MetadataResponse_v0
<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]> Response 1 (110.08358001708984 ms): ApiVersionResponse_v0(error_code=0, api_versions=[(api_key=0, min_version=0, max_version=8), (api_key=1, min_version=0, max_version=11), (api_key=2, min_version=0, max_version=5), (api_key=3, min_version=0, max_version=9), (api_key=4, min_version=0, max_version=4), (api_key=5, min_version=0, max_version=2), (api_key=6, min_version=0, max_version=6), (api_key=7, min_version=0, max_version=3), (api_key=8, min_version=0, max_version=8), (api_key=9, min_version=0, max_version=6), (api_key=10, min_version=0, max_version=3), (api_key=11, min_ver

Added sensor with name heartbeat-latency
Added sensor with name join-latency
Added sensor with name sync-latency
group_id is None: disabling auto-commit.
Added sensor with name commit-latency
Seeking to end of partition TopicPartition(topic='eolienne_jour_1', partition=0)
Resetting offset for partition TopicPartition(topic='eolienne_jour_1', partition=0) to latest offset.
Partition TopicPartition(topic='eolienne_jour_1', partition=0) is unknown for fetching offset, wait for metadata refresh
Sending metadata request MetadataRequest_v1(topics=['eolienne_jour_1']) to node bootstrap-0
Sending request MetadataRequest_v1(topics=['eolienne_jour_1'])
<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]> Request 3: MetadataRequest_v1(topics=['eolienne_jour_1'])
Received correlation id: 3
Processing response MetadataResponse_v1
<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]> Response 3 (3.03435325622558

14611


Advance position for partition TopicPartition(topic='eolienne_jour_1', partition=0) from 9106 to 9236 (last message batch location plus one) to correct for deleted compacted messages
Adding fetch request for partition TopicPartition(topic='eolienne_jour_1', partition=0) at offset 9236
Sending FetchRequest to node 0
Sending request FetchRequest_v4(replica_id=-1, max_wait_time=500, min_bytes=1, max_bytes=52428800, isolation_level=0, topics=[(topic='eolienne_jour_1', partitions=[(partition=0, offset=9236, max_bytes=1048576)])])
<BrokerConnection node_id=0 host=irlin328206:9092 <connected> [IPv6 ('fe80::9a90:96ff:fead:e31d', 9092, 0, 2)]> Request 5: FetchRequest_v4(replica_id=-1, max_wait_time=500, min_bytes=1, max_bytes=52428800, isolation_level=0, topics=[(topic='eolienne_jour_1', partitions=[(partition=0, offset=9236, max_bytes=1048576)])])
Received correlation id: 5
Processing response FetchResponse_v4
<BrokerConnection node_id=0 host=irlin328206:9092 <connected> [IPv6 ('fe80::9a90:96f

14610


http://localhost:9086 "POST /write?db=test&rp=test_policy HTTP/1.1" 204 0
Reporting span 37fbada774797cad:4e9096fd5c448b32:1f9c83ffd2381f25:1 influxdb_test_insert_eolienne_1_jour.insert_bulk
Reporting span 37fbada774797cad:1f9c83ffd2381f25:0:1 influxdb_test_insert_eolienne_1_jour.test_bulk_insertion


In [3]:
client.get_list_database()

http://localhost:9086 "GET /query?q=SHOW+DATABASES&db=test HTTP/1.1" 200 None


[{'name': '_internal'}, {'name': 'test'}]

In [4]:
client.get_list_retention_policies()

http://localhost:9086 "GET /query?q=SHOW+RETENTION+POLICIES+ON+%22test%22&db=test HTTP/1.1" 200 None


[{'name': 'autogen',
  'duration': '0s',
  'shardGroupDuration': '168h0m0s',
  'replicaN': 1,
  'default': True},
 {'name': 'teeth',
  'duration': '0s',
  'shardGroupDuration': '1h50m0s',
  'replicaN': 1,
  'default': False},
 {'name': 'test_policy',
  'duration': '0s',
  'shardGroupDuration': '1h50m0s',
  'replicaN': 1,
  'default': False}]

In [5]:
client.get_list_measurements()

http://localhost:9086 "GET /query?q=SHOW+MEASUREMENTS&db=test HTTP/1.1" 200 None


[{'name': 'brushEvents'}, {'name': 'smartGrid'}]

In [5]:
results = client.query('SELECT * FROM "test_policy"."smartGrid"')
results.raw

http://localhost:9086 "GET /query?q=SELECT+%2A+FROM+%22test_policy%22.%22smartGrid%22&db=test HTTP/1.1" 200 None


{'statement_id': 0}

In [4]:
d = client.delete_series("test","smartGrid")
d

http://localhost:9086 "POST /query?q=DROP+SERIES+FROM+%22smartGrid%22&db=test HTTP/1.1" 200 None
